# Grasple Test to Microsoft Excel

o,porty Below you can upload a `.csv`-file exported from a Grasple test. The file will be converted to a Microsoft Excel file, which downloads automatically after generation.

In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
import io
import numpy as np

# Widgets
upload = widgets.FileUpload(accept='.csv', multiple=False)
output = widgets.Output()

def on_upload_change(change):
    output.clear_output()
    if upload.value:
        with output:
            try:
                # Retrieve the first file from the tuple
                uploaded_file = upload.value[0] if isinstance(upload.value, tuple) else list(upload.value.values())[0]
                
                # In some environments, upload.value[0] is a dictionary.
                if isinstance(uploaded_file, dict):
                    content = uploaded_file['content']
                else:
                    # Sometimes it's a tuple (name, file dict)
                    content = uploaded_file[1]['content']
                
                df = pd.read_csv(io.BytesIO(content))
                df = df.drop(df[df['started_test'] == 0].index)
                df['main_exercise_id'] = df['main_exercise_id'].astype(int)
                print("✅ CSV successfully loaded.")
                
                # Convert to pivot table
                # Extract the unique question pool names from the column names and determine whether question pools are used
                question_pool_names_temp = df['question_pool_name'].unique()
                question_pool_names = []
                for name in question_pool_names_temp:
                    if isinstance(name, str):
                        question_pool_names.append(name)
                
                if len(question_pool_names) <= 0:
                    question_pool_names = df['main_exercise_id'].unique()
                pvt = pd.pivot_table(df, values="scored_points", index="student_id", aggfunc="sum", fill_value=0, columns="main_exercise_id")
                
                display(pvt)
            except Exception as e:
                print("❌ Error loading CSV:")
                print(e)

upload.observe(on_upload_change, names='value')

# Display the widgets
display(upload, output)
